# Apresentação do Projeto

### Blibliotecas

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

### Bases

In [2]:
dim_circuits = pd.read_csv(
    "https://raw.githubusercontent.com/AbilioNB/datawizards/main/DW/DIM_CIRCUITS.csv")
dim_constructors = pd.read_csv(
    "https://raw.githubusercontent.com/AbilioNB/datawizards/main/DW/DIM_CONSTRUCTORS.csv")
dim_drivers = pd.read_csv(
    "https://raw.githubusercontent.com/AbilioNB/datawizards/main/DW/DIM_DRIVERS.csv")
dim_races = pd.read_csv(
    "https://raw.githubusercontent.com/AbilioNB/datawizards/main/DW/DIM_RACES.csv")
dim_date = pd.read_csv(
    "https://raw.githubusercontent.com/AbilioNB/datawizards/main/DW/DIM_DATE.csv")
fact_results = pd.read_csv(
    "https://raw.githubusercontent.com/AbilioNB/datawizards/main/DW/FACT_RESULTS.csv")

## Realizando o processamento da base

---



### Unindo as bases 

In [3]:
df_fact = fact_results.merge(
    dim_races, how='inner', on='RACE_ID', suffixes=[None, None])
df_fact = df_fact.merge(dim_circuits, how='inner',
                        on='CIRCUIT_ID', suffixes=[None, None])
df_fact = df_fact.merge(dim_drivers, how='inner',
                        on='DRIVER_ID', suffixes=[None, None])
df_fact = df_fact.merge(dim_constructors, how='inner',
                        on='CONSTRUCTOR_ID', suffixes=[None, None])
df_fact = df_fact.merge(dim_date, how='inner',
                        on='DATE_ID', suffixes=[None, None])

# Funções

In [4]:
# PILOTOS COM MAIS VITÓRIAS NO STANDINGS POR ANO

def driver_points_sum(df, year):
    driver_points_sum = df[df['YEAR'] == year].groupby(
        ['DRIVER_NAME', 'CONSTRUCTOR_NAME'])['RACE_DRIVER_POINTS'].sum().reset_index()
    driver_points_sum = driver_points_sum.sort_values(
        by=['RACE_DRIVER_POINTS'], ascending=False).reset_index(drop=True)
    return driver_points_sum

# CONSTRUTORAS COM MAIS VITÓRIAS NO STANDINGS POR ANO


def constructor_standings(df, year):

    constructor_wins = df[df['YEAR'] == year].groupby(['CONSTRUCTOR_NAME']).agg(
        {'CONSTRUCTOR_RESULTS_POINTS': 'sum', 'DRIVER_NAME': 'nunique'}).reset_index()
    constructor_wins['CONSTRUCTOR_RESULTS_POINTS'] = constructor_wins['CONSTRUCTOR_RESULTS_POINTS'].divide(
        constructor_wins['DRIVER_NAME'], axis=0).astype(int)
    constructor_wins = constructor_wins.sort_values(
        by='CONSTRUCTOR_RESULTS_POINTS', ascending=False)
    constructor_wins = constructor_wins.drop(['DRIVER_NAME'], axis=1).rename(
        columns={'CONSTRUCTOR_RESULTS_POINTS': 'POINTS'})
    constructor_wins = constructor_wins.reset_index(drop=True)
    return constructor_wins

# PILOTOS COM MAIS POLE POSITIONS


def driver_pole_count(df, year=''):
    if year == '':
        driver_pole = df[df['GRID_POSITION'] == 1]
        driver_pole = driver_pole.groupby(
            'DRIVER_NAME')['GRID_POSITION'].count().reset_index()
        driver_pole = driver_pole.sort_values(
            by='GRID_POSITION', ascending=False)
    else:
        driver_pole = df[(df['YEAR'] == year) & (df['GRID_POSITION'] == 1)]
        driver_pole = driver_pole.groupby(
            'DRIVER_NAME')['GRID_POSITION'].count().reset_index()
        driver_pole = driver_pole.sort_values(
            by='GRID_POSITION', ascending=False)
    return driver_pole.rename(columns={'GRID_POSITION': 'POLE_COUNT'}).reset_index(drop=True)

# PILOTOS COM MAIS PÓDIOS


def driver_podium_count(df, year=''):
    if year == '':
        drivers_podium = df[df['DRIVER_FINISH_POSITION'] <= 3]
        drivers_podium = drivers_podium.groupby(
            'DRIVER_NAME')['DRIVER_FINISH_POSITION'].count().reset_index()
        drivers_podium = drivers_podium.sort_values(
            by='DRIVER_FINISH_POSITION', ascending=False)
    else:
        drivers_podium = df[(df['DRIVER_FINISH_POSITION']
                             <= 3) & (df_fact['YEAR'] == year)]
        drivers_podium = drivers_podium.groupby(
            'DRIVER_NAME')['DRIVER_FINISH_POSITION'].count().reset_index()
        drivers_podium = drivers_podium.sort_values(
            by='DRIVER_FINISH_POSITION', ascending=False)
    return drivers_podium.rename(columns={'DRIVER_FINISH_POSITION': 'PODIUM_FINISH'}).reset_index(drop=True)

# PILOTOS COM MAIS VITÓRIAS EM CORRIDAS


def driver_win_count(df, year=''):
    if year == '':
        driver_win_count = df[df['DRIVER_FINISH_POSITION'] == 1]
        driver_win_count = driver_win_count.groupby(
            'DRIVER_NAME')['DRIVER_FINISH_POSITION'].count().reset_index()
        driver_win_count = driver_win_count.sort_values(
            by='DRIVER_FINISH_POSITION', ascending=False)
    else:
        driver_win_count = df[(df['DRIVER_FINISH_POSITION']
                               == 1) & (df['YEAR'] == year)]
        driver_win_count = driver_win_count.groupby(
            'DRIVER_NAME')['DRIVER_FINISH_POSITION'].count().reset_index()
        driver_win_count = driver_win_count.sort_values(
            by='DRIVER_FINISH_POSITION', ascending=False)
        driver_win_count = driver_win_count.rename(
            columns={"DRIVER_FINISH_POSITION": "WINS"})
    return driver_win_count

# CLASSIFICAÇÃO DO CAMPEONATO DE PILOTOS


def driver_standings(df, year):
    points = driver_points_sum(df, year)
    pole = driver_pole_count(df, year)
    podium = driver_podium_count(df, year)
    win = driver_win_count(df, year)
    stand = points.merge(pole, how='outer', on='DRIVER_NAME').merge(
        win, how='outer', on='DRIVER_NAME').merge(podium, how='outer', on='DRIVER_NAME')
    stand = stand.rename(columns={
        "CONSTRUCTOR_NAME": "CONSTRUCTOR",
        "RACE_DRIVER_POINTS": "POINTS",
        "GRID_POSITION": "POLE_POSITIONS",
        "DRIVER_FINISH_POSITION": "WINS",
        "DRIVER_FINISH_POSITION_y": "PODIUMS"})
    stand.fillna(0, inplace=True)
    return stand

# Perguntas do desafio:



1.   Classificação do campeonado de pilotos
2.   Classificação do campeonado de contrutores
3.   Pilotos com mais pódios
4.   Pilotos com mais poles (primeiro lugar no treino classificatório)




In [5]:
year = 2020
df = df_fact

### 1.Classificação do campeonado de pilotos


In [6]:
print(driver_standings(df, year))
fig = px.bar(driver_standings(df, year).head(20), color='WINS', x='DRIVER_NAME',
             y='POINTS', labels={"DRIVER_NAME": "Piloto", "POINTS": "Pontos"})
fig.show()

           DRIVER_NAME   CONSTRUCTOR  POINTS  POLE_COUNT  WINS  PODIUM_FINISH
0       Lewis Hamilton      Mercedes   347.0        10.0  11.0           14.0
1      Valtteri Bottas      Mercedes   223.0         5.0   2.0           11.0
2       Max Verstappen      Red Bull   214.0         1.0   2.0           11.0
3         Sergio Pérez  Racing Point   125.0         0.0   1.0            2.0
4     Daniel Ricciardo       Renault   119.0         0.0   0.0            2.0
5      Alexander Albon      Red Bull   105.0         0.0   0.0            2.0
6         Carlos Sainz       McLaren   105.0         0.0   0.0            1.0
7      Charles Leclerc       Ferrari    98.0         0.0   0.0            2.0
8         Lando Norris       McLaren    97.0         0.0   0.0            1.0
9         Pierre Gasly    AlphaTauri    75.0         0.0   1.0            1.0
10        Lance Stroll  Racing Point    75.0         1.0   0.0            2.0
11        Esteban Ocon       Renault    62.0         0.0   0.0  

### 2.Classificação do campeonado de contrutores

In [7]:
print(constructor_standings(df, year))
fig = px.bar(constructor_standings(df, year).head(10), x='CONSTRUCTOR_NAME', y='POINTS', labels={
             "CONSTRUCTOR_NAME": "Construtores", "POINTS": "Pontuação dos Construtores"})
fig.show()

  CONSTRUCTOR_NAME  POINTS
0         Mercedes     382
1         Red Bull     319
2          McLaren     202
3          Renault     181
4     Racing Point     140
5          Ferrari     131
6       AlphaTauri     107
7       Alfa Romeo       8
8     Haas F1 Team       2
9         Williams       0


### 3.Pilotos com mais pódios

In [8]:
print(driver_podium_count(df, 2020).head(10))
fig = px.bar(driver_podium_count(df, 2020).head(10), x='DRIVER_NAME', y='PODIUM_FINISH',
             labels={"DRIVER_NAME": "Piloto", "PODIUM_FINISH": "Quantidade de Podium"})
fig.show()

        DRIVER_NAME  PODIUM_FINISH
0    Lewis Hamilton             14
1    Max Verstappen             11
2   Valtteri Bottas             11
3   Alexander Albon              2
4   Charles Leclerc              2
5  Daniel Ricciardo              2
6      Lance Stroll              2
7      Sergio Pérez              2
8      Carlos Sainz              1
9      Esteban Ocon              1


### 4.Pilotos com mais poles (primeiro lugar no treino classificatório)

In [13]:
print(driver_pole_count(df).head(10))
fig = px.bar(driver_pole_count(df).head(10), x='DRIVER_NAME', y='POLE_COUNT',
             labels={"DRIVER_NAME": "Piloto", "POLE_COUNT": "Quantidade de Pole"})
fig.show()

          DRIVER_NAME  POLE_COUNT
0      Lewis Hamilton         100
1  Michael Schumacher          68
2        Ayrton Senna          65
3    Sebastian Vettel          57
4         Alain Prost          33
5       Nigel Mansell          32
6        Nico Rosberg          30
7       Mika Häkkinen          26
8          Niki Lauda          24
9       Nelson Piquet          24
